In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

Using TensorFlow backend.


In [27]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [ ]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [28]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 9, 9, 1920)

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [29]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [30]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [31]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [32]:
samples = samples.reshape(nos,9*9*1920)

In [33]:
samples.shape, labels.shape

((472, 155520), (472,))

In [34]:
# To save memory
del X
del y

In [35]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 2s 4ms/step - loss: 13.3725 - accuracy: 0.6465 - val_loss: 7.0770 - val_accuracy: 0.7627
Epoch 2/100
413/413 [==============================] - 1s 3ms/step - loss: 4.9535 - accuracy: 0.7966 - val_loss: 1.9910 - val_accuracy: 0.7966
Epoch 3/100
413/413 [==============================] - 1s 3ms/step - loss: 0.8236 - accuracy: 0.8765 - val_loss: 0.9614 - val_accuracy: 0.7627
Epoch 4/100
413/413 [==============================] - 1s 3ms/step - loss: 0.3586 - accuracy: 0.9080 - val_loss: 0.5083 - val_accuracy: 0.7797
Epoch 5/100
413/413 [==============================] - 1s 3ms/step - loss: 0.1877 - accuracy: 0.9467 - val_loss: 0.4626 - val_accuracy: 0.7797
Epoch 6/100
413/413 [==============================] - 1s 3ms/step - loss: 0.0887 - accuracy: 0.9637 - val_loss: 0.4331 - val_accuracy: 0.7966
Epoch 7/100
413/413 [==============================] - 1s 3ms/step - loss: 0.0821 - accur

In [36]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.8612068965517242 0.8178160919540232


Training on whole Training Dataset

In [42]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [43]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [44]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [45]:
datagentest.fit(xtest)

In [46]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,1920))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 1920)


In [47]:
test_samples = test_samples.reshape(testnos,9*9*1920)

In [48]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 2s 5ms/step - loss: 9.8908 - accuracy: 0.6695 - val_loss: 1.8891 - val_accuracy: 0.7953
Epoch 2/100
472/472 [==============================] - 2s 4ms/step - loss: 1.1388 - accuracy: 0.8496 - val_loss: 0.5648 - val_accuracy: 0.8931
Epoch 3/100
472/472 [==============================] - 2s 4ms/step - loss: 0.3929 - accuracy: 0.8644 - val_loss: 0.5236 - val_accuracy: 0.8442
Epoch 4/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2319 - accuracy: 0.9174 - val_loss: 0.5788 - val_accuracy: 0.7899
Epoch 5/100
472/472 [==============================] - 2s 4ms/step - loss: 0.1546 - accuracy: 0.9407 - val_loss: 0.7173 - val_accuracy: 0.7935
Epoch 6/100
472/472 [==============================] - 2s 4ms/step - loss: 0.1679 - accuracy: 0.9555 - val_loss: 0.5883 - val_accuracy: 0.8134
Epoch 7/100
472/472 [==============================] - 2s 4ms/step - loss: 0.0986 - accuracy: 0.

In [49]:
test_preds = model.predict(test_samples, batch_size = 20)

In [50]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [51]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.6779661016949152, 0.8762677484787018)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

TypeError: ignored